In [1]:
from tensorflow import keras
from keras import applications
from keras import optimizers
from keras.models import Sequential,Model
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

In [91]:
data_path = "/home/kritika/Downloads/FYP/dataset/GTSRB/pre-processed_images/"
train_datagen = ImageDataGenerator(
        featurewise_center=False, 
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        zca_epsilon=1e-06,
        rotation_range=0,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.,
        zoom_range=0.,
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,  
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    "/home/kritika/Downloads/FYP/dataset/GTSRB/pre-processed_images/",
    target_size=(32,32),
    batch_size=32,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    "/home/kritika/Downloads/FYP/dataset/GTSRB/pre-processed_images/", # same directory as training data
    target_size=(32,32),
    batch_size=32,
    class_mode="categorical",
    subset='validation') # set as validation data


Found 5228 images belonging to 43 classes.
Found 1307 images belonging to 43 classes.


In [56]:
model = Sequential()
"""
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
"""
model.add(applications.VGG16(include_top=False, pooling='avg', weights="imagenet",input_shape=(32,32,3)))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(43, activation='softmax'))
print(model.summary())

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 512)               14714688  
_________________________________________________________________
dense_31 (Dense)             (None, 128)               65664     
_________________________________________________________________
activation_21 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_23 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_32 (Dense)             (None, 64)                8256      
_________________________________________________________________
activation_22 (Activation)   (None, 64)                0         
_________________________________________________________________
dropout_24 (Dropout)         (None, 64)              

In [57]:
model.input

<KerasTensor: shape=(None, 32, 32, 3) dtype=float32 (created by layer 'vgg16_input')>

In [58]:
model.layers[0].trainable = False
optimizer = optimizers.Adam(lr=0.0001,
                  beta_1=0.9,
                  beta_2=0.999,
                  epsilon=1e-08,
                  )
model.compile(loss="categorical_crossentropy",
              optimizer=optimizer,
              metrics=["categorical_accuracy"]) 
#model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [59]:
model.fit_generator(train_generator,
        steps_per_epoch=10,
        validation_data=validation_generator,
        validation_steps=10,
        epochs=20, verbose=1)

Epoch 1/20
10/10 [==============================] - 5s 425ms/step - loss: 42.9344 - categorical_accuracy: 0.0283 - val_loss: 13.9543 - val_categorical_accuracy: 0.0156
Epoch 2/20
10/10 [==============================] - 4s 404ms/step - loss: 36.1914 - categorical_accuracy: 0.0356 - val_loss: 12.6194 - val_categorical_accuracy: 0.0188
Epoch 3/20
10/10 [==============================] - 4s 421ms/step - loss: 38.9057 - categorical_accuracy: 0.0104 - val_loss: 11.3343 - val_categorical_accuracy: 0.0250
Epoch 4/20
10/10 [==============================] - 4s 383ms/step - loss: 30.5798 - categorical_accuracy: 0.0265 - val_loss: 8.8841 - val_categorical_accuracy: 0.0406
Epoch 5/20
10/10 [==============================] - 4s 383ms/step - loss: 30.9673 - categorical_accuracy: 0.0183 - val_loss: 8.4327 - val_categorical_accuracy: 0.0344
Epoch 6/20
10/10 [==============================] - 4s 391ms/step - loss: 26.0103 - categorical_accuracy: 0.0359 - val_loss: 7.3234 - val_categorical_accuracy: 0.

In [69]:
def resnet_v2(input_shape, depth, num_classes=43):
    
    #if (depth - 2) % 9 != 0:
    #    raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [73]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


In [74]:
batch_size = 32  # orig paper trained all networks with batch_size=128
epochs = 10
data_augmentation = True
num_classes = 43

In [80]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10

In [82]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr



In [83]:
depth = 18

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

input_shape = (32,32,3)
model = resnet_v2(input_shape=input_shape, depth=depth)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
print(model_type)

Learning rate:  0.001
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 32, 32, 16)   448         input_11[0][0]                   
__________________________________________________________________________________________________
batch_normalization_10 (BatchNo (None, 32, 32, 16)   64          conv2d_15[0][0]                  
__________________________________________________________________________________________________
activation_34 (Activation)      (None, 32, 32, 16)   0           batch_normalization_10[0][0]     
______________________________________________________________________

In [92]:
"""model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=10,
              validation_data=(x_test, y_test),
              shuffle=True
              )"""

model.fit_generator(train_generator,
        steps_per_epoch=10,
        validation_data=validation_generator,
        validation_steps=10,
        epochs=10, verbose=1)

Epoch 1/10
10/10 [==============================] - 3s 291ms/step - loss: 3.1711 - accuracy: 0.1906 - val_loss: 4.5269 - val_accuracy: 0.0938
Epoch 2/10
10/10 [==============================] - 3s 272ms/step - loss: 3.1473 - accuracy: 0.2031 - val_loss: 4.4137 - val_accuracy: 0.1219
Epoch 3/10
10/10 [==============================] - 3s 274ms/step - loss: 2.9431 - accuracy: 0.2281 - val_loss: 4.2709 - val_accuracy: 0.1063
Epoch 4/10
10/10 [==============================] - 3s 275ms/step - loss: 2.9905 - accuracy: 0.2281 - val_loss: 3.8069 - val_accuracy: 0.1375
Epoch 5/10
10/10 [==============================] - 3s 285ms/step - loss: 2.8849 - accuracy: 0.2688 - val_loss: 4.0323 - val_accuracy: 0.0938
Epoch 6/10
10/10 [==============================] - 3s 274ms/step - loss: 2.7768 - accuracy: 0.2937 - val_loss: 4.8353 - val_accuracy: 0.1094
Epoch 7/10
10/10 [==============================] - 3s 277ms/step - loss: 2.7804 - accuracy: 0.3000 - val_loss: 4.3059 - val_accuracy: 0.1000
Epoch 